In [ ]:
from google.colab import drive
import pandas as pd
import torch
drive.mount('/content/drive')
# Pfad zur CSV-Datei in Google Drive
file_path = '/content/drive/My Drive/combined_data.csv'

# Einlesen der CSV-Datei
data = pd.read_csv(file_path)

print(torch.cuda.is_available())


Mounted at /content/drive
True


In [ ]:
''''
Alle Probanden  (Sqaut und Jump Bewegungen)
'''

import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
import numpy as np

# Daten laden
#data = pd.read_csv(r'C:\Users\badrl\OneDrive - ZHAW\Desktop\combined_data.csv')
data = data[data['Exercise'].isin(['Squat', 'Jump'])]

# Entfernen von 'Timestamp' und 'Subject'
data = data.drop(columns=['Timestamp', 'Subject'])

# One-Hot-Encoding für 'Exercise'
data = pd.get_dummies(data, columns=['Exercise'])
# Normalisierung der numerischen Daten
# Erstellen eines Scalers
scaler = StandardScaler()
# Wählen Sie die numerischen Spalten aus
numeric_columns = data.select_dtypes(include=['float64', 'int64','bool']).columns
# Anwenden des Scalers auf die numerischen Spalten
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

# Umwandlung der Daten in Sequenzen von je 240 Zeilen
sequences = [data.iloc[i:i+240] for i in range(0, len(data), 240)]

# Stellen Sie sicher, dass die letzte Sequenz vollständig ist
sequences = [seq for seq in sequences if len(seq) == 240]

# Umwandlung jeder Sequenz in einen Tensor
tensor_sequences = [torch.tensor(sequence.values).float() for sequence in sequences]


In [ ]:
'''
Transformer-Autoencoder
'''
#Libraries
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
import random
import numpy as np
import torch.nn.functional as F
import math
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.interpolate import make_interp_spline
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.spatial.distance import cdist
import seaborn as sns
import onnx




class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500, device=torch.device('cpu')):
        super().__init__()
        # Erstellen einer Positionskodierungsmatrix
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1).to(device)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)).to(device)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1).to(device)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # Addiert die Positionskodierung zur Eingabe
        x = x + self.pe[:x.size(0), :]
        return x

class TransformerAutoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim, num_layers=8):
        super(TransformerAutoencoder, self).__init__()
        # Initialisierung der Klassenvariablen und der Positionskodierung

        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.positional_encoder = PositionalEncoding(d_model=input_dim)

        # Lineare Reduktion der Dimensionen mit Normalisierung
        self.linear_reduction = nn.Linear(input_dim, latent_dim)
        self.linear_reduction_norm = nn.LayerNorm(latent_dim)

        # Encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=latent_dim, nhead=2, norm_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Decoder
        decoder_layer = nn.TransformerDecoderLayer(d_model=latent_dim, nhead=2, norm_first=True)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

        # Zusätzliche lineare Schichten und Normalisierung
        self.linear1 = nn.Linear(latent_dim, latent_dim * 2)
        self.linear1_norm = nn.LayerNorm(latent_dim * 2)  # Layer-Normalisierung hinzugefügt
        self.linear2 = nn.Linear(latent_dim * 2, latent_dim)
        self.linear2_norm = nn.LayerNorm(latent_dim)  # Layer-Normalisierung hinzugefügt

        # Rekonstruktions-Output
        self.decoder_output = nn.Linear(latent_dim, input_dim)


    def encode(self, x):
        # Kodierungsprozess: Positionskodierung, lineare Reduktion und Durchlauf durch den Transformer Encoder
        x = self.positional_encoder(x)
        x = self.linear_reduction(x)
        x = self.linear_reduction_norm(x)
        encoded_x = self.transformer_encoder(x)
        return encoded_x

    def decode(self, encoded_x):
      # Dekodierungsprozess: Durchlauf durch den Transformer Decoder und lineare Transformation
      decoded_x = self.transformer_decoder(encoded_x, torch.zeros_like(encoded_x))
      decoded_x = F.relu(self.linear1(decoded_x))
      decoded_x = self.linear2(decoded_x)
      decoded_x = self.decoder_output(decoded_x)
      return decoded_x

    def forward(self, x):
        # Gesamter Prozess: Kodieren und Dekodieren
        encoded_x = self.encode(x)
        recon_x = self.decode(encoded_x)
        return recon_x

# Verlustfunktion
def loss_function(recon_x, x, delta=1):
    recon_loss = F.smooth_l1_loss(recon_x, x, reduction='mean', beta=delta)
    return recon_loss

#Model Trainieren
def pretrain_autoencoder(model, train_loader, val_loader, optimizer, epochs):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        # Training
        model.train()
        total_train_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            reconstructed = model(data)
            loss = loss_function(reconstructed, data)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        train_losses.append(total_train_loss)

        # Validierung
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for data in val_loader:
                data = data.to(device)
                reconstructed = model(data)
                loss = loss_function(reconstructed, data)
                total_val_loss += loss.item()
        val_losses.append(total_val_loss)

        print(f"Epoch {epoch}, Train Loss: {total_train_loss}, Val Loss: {total_val_loss}")


        if epoch == epochs - 1:
            # Extrahieren und Rekonstruieren einer Sequenz aus dem Trainingsset
            train_example = next(iter(train_loader))
            train_seq = train_example.to(device)
            recon_train_seq = model(train_seq)

            # Extrahieren und Rekonstruieren einer Sequenz aus dem Validierungsset
            val_example = next(iter(val_loader))
            val_seq = val_example.to(device)
            recon_val_seq = model(val_seq)

            # Plotten des Durchschnitts der Features von Trainings- und Validierungsset
            plot_average_feature_comparison(train_seq[0].detach().cpu().numpy(), recon_train_seq[0].detach().cpu().numpy(), 'Vergleich von originalen und rekonstruirten Trainigsdaten')
            plot_average_feature_comparison(val_seq[0].detach().cpu().numpy(), recon_val_seq[0].detach().cpu().numpy(), 'Vergleich von originalen und rekonstruirten Validierungsdaten')

            recon_error_train = loss_function(recon_train_seq, train_seq).item()
            recon_error_val = loss_function(recon_val_seq, val_seq).item()

    plot_train_val_loss(train_losses, val_losses)


# Losses plotten
def plot_train_val_loss(train_losses, val_losses):
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Trainingsverlust')
    plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validierungsverlust')
    plt.title('Trainings- und Validierungsverlust über die Epochen')
    plt.xlabel('Epochen')
    plt.ylabel('Verlust')
    plt.legend()
    plt.grid(True)
    wandb.log({"Trainings- und Validierungsverlust über die Epochen": wandb.Image(plt.gcf())})
    plt.show()

#Originale und Rekonstruirten Daten Plotten
def plot_average_feature_comparison(original_data, reconstructed_data, title):
    """Funktion zum Plotten des Durchschnitts über alle Features."""
    avg_original = np.mean(original_data, axis=1)
    avg_reconstructed = np.mean(reconstructed_data, axis=1)
    plt.figure(figsize=(15, 5))
    plt.plot(avg_original, label='Originale Daten')
    plt.plot(avg_reconstructed, label=' Rekonstruirte Daten', alpha=0.7)
    plt.title(title)
    plt.xlabel('Sequenzlänge')
    plt.ylabel('Durchschnittlicher Merkmalswert')
    plt.legend()
    wandb.log({"Original Data-Reconstructed data": wandb.Image(plt.gcf())})
    plt.show()


#Visualisierung der Datenverteilung von Normale und Anomale
def visualize_data_distribution(model, train_loader, test_loader, device=torch.device('cpu'), percentile=95, num_features_to_plot=5):
    model.to(device).eval()

    # Berechnung des Schwellenwerts für den Rekonstruktionsfehler
    train_errors = []
    with torch.no_grad():
        for batch in train_loader:
            batch = batch.to(device)
            reconstructed = model(batch)
            batch_error = torch.mean((batch - reconstructed) ** 2, dim=[1, 2])
            train_errors.extend(batch_error.cpu().numpy())
    threshold = np.percentile(train_errors, percentile)

    # Klassifizierung der Testdaten als normal oder anomal
    normal_data = []
    anomalous_data = []
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            reconstructed = model(batch)
            batch_error = torch.mean((batch - reconstructed) ** 2, dim=[1, 2])

            for i in range(batch.size(0)):
                if batch_error[i].item() > threshold:
                    anomalous_data.append(batch[i].cpu().numpy())
                else:
                    normal_data.append(batch[i].cpu().numpy())

    # Umwandeln in NumPy-Arrays
    normal_data = np.array(normal_data).reshape(-1, batch.shape[2])  # Flattening der Sequenzen
    anomalous_data = np.array(anomalous_data).reshape(-1, batch.shape[2])

    # Visualisierung der Datenverteilung für ausgewählte Features
    for i in range(min(num_features_to_plot, batch.shape[2])):
        plt.figure(figsize=(10, 4))
        sns.histplot(normal_data[:, i], color="blue", kde=True, stat="density", linewidth=0, label="Normal")
        sns.histplot(anomalous_data[:, i], color="red", kde=True, stat="density", linewidth=0, label="Anomal")
        plt.title(f'Verteilung des Features {i}')
        plt.legend()
        wandb.log({"Verteilung des Features": wandb.Image(plt.gcf())})
        plt.show()


def anomalies_identification_with_ReconstructionError(model, train_loader, test_loader, percentile=95, device=torch.device('cpu')):
    model.to(device).eval()

    # Berechnung des Schwellenwerts basierend auf den Trainingsdaten
    train_errors = []
    with torch.no_grad():
        for batch in train_loader:
            batch = batch.to(device)
            reconstructed = model(batch)
            batch_error = F.smooth_l1_loss(reconstructed, batch, reduction='none')
            batch_error = batch_error.view(batch.size(0), -1).mean(dim=1)
            train_errors.extend(batch_error.cpu().numpy())
    threshold = np.percentile(train_errors, percentile)

    # Berechnung der Rekonstruktionsfehler für die Testdaten
    test_errors = []
    anomalies = []
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            reconstructed = model(batch)
            for i in range(batch.size(0)):
                error = F.smooth_l1_loss(reconstructed[i], batch[i], reduction='mean').item()
                test_errors.append(error)
                anomalies.append(error > threshold)

    # Trennung der Fehler in normal und anomal
    normal_errors = np.array(test_errors)[~np.array(anomalies)]
    anomalous_errors = np.array(test_errors)[np.array(anomalies)]

    # Plotten der Fehler
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    plt.scatter(range(len(test_errors)), test_errors, c=anomalies, cmap='coolwarm', label='Rekonstruktionsfehler')
    plt.axhline(y=threshold, color='r', linestyle='--', label='Schwellenwert')
    plt.xlabel('Sequenzindex')
    plt.ylabel('Rekonstruktionsfehler')
    plt.title('Rekonstruktionsfehler pro Sequenz (Bewegung)')
    plt.legend()

    # Erstellen von DataFrames für normale und anomale Daten
    df_normal = pd.DataFrame({'Fehler': normal_errors, 'Typ': 'Normal'})
    df_anomal = pd.DataFrame({'Fehler': anomalous_errors, 'Typ': 'Anomal'})

    # Zusammenführen der DataFrames
    df_combined = pd.concat([df_normal, df_anomal])

    # Statistische Vergleiche
    plt.subplot(1, 2, 2)
    sns.boxplot(x='Typ', y='Fehler', data=df_combined)
    plt.title('Vergleich der Fehlerstatistiken')
    plt.ylabel('Rekonstruktionsfehler')
    wandb.log({"Vergleich der Fehlerstatistiken": wandb.Image(plt.gcf())})
    plt.show()

    # Identifizieren von Anomalien
    print(f"Identifizierte Anomalien in den Sequenzen: {np.where(anomalies)[0]}")
    print("Anzahl Anomalien: ", sum(anomalies))
    visualize_data_distribution(model, train_loader, test_loader, device=device, percentile=percentile)

    wandb.log({"Anomalien Identifikation mit Rekonstruktionsfehler": wandb.Image(plt.gcf())})


#Berechnung den Durchschnittlichen Rekonstruktionsfehler
def calculate_average_reconstruction_error(model, data_loader, device=torch.device('cpu')):
    model.to(device).eval()
    total_error = 0.0

    with torch.no_grad():
        for data in data_loader:
            data = data.to(device)
            reconstructed = model(data)
            error = F.smooth_l1_loss(reconstructed, data, reduction='mean').item()  # Summe der Fehler
            total_error += error
    return total_error

#Extraktion latenter Repräsentationen
def extract_latent_representations(dataloader, model):
    model.eval()
    latent_representations = []
    with torch.no_grad():
        for batch in dataloader:
            data = batch.to(device)
            latent = model.encode(data)  # Latente Repräsentation
            if latent.dim() > 2:
                latent = latent.mean(dim=1)
            latent_representations.append(latent.cpu().numpy())
    return np.concatenate(latent_representations)

#K-Means Clustering auf latenten Repräsentationen
def anomalies_identification_with_Kmeans(train_loader, test_loader, autoencoder):
    # Extrahieren der latenten Repräsentationen
    latent_representations_train = extract_latent_representations(train_loader, autoencoder)
    latent_representations_test = extract_latent_representations(test_loader, autoencoder)

    # K-Means Clustering auf den Trainingsdaten
    kmeans = KMeans(n_clusters=2, n_init=2, random_state=0)
    kmeans.fit(latent_representations_train)


    # Konvertieren der Clusterzentren in einen PyTorch-Tensor
    cluster_centers_tensor = torch.tensor(kmeans.cluster_centers_, dtype=torch.float32)

    # Speichern des Tensors
    torch.save(cluster_centers_tensor, 'cluster_centers.pt')

    # PCA für Reduzierung auf 2 Dimensionen
    pca = PCA(n_components=2)
    latent_pca_train = pca.fit_transform(latent_representations_train)
    latent_pca_test = pca.transform(latent_representations_test)
    cluster_centers_pca = pca.transform(kmeans.cluster_centers_)

    # Zuordnung und Distanzberechnung für Testdaten
    test_cluster_labels = kmeans.predict(latent_representations_test)
    dists_to_centers = np.min(cdist(latent_representations_test, kmeans.cluster_centers_), axis=1)

    # Schwellenwert für Anomalien
    threshold = np.percentile(dists_to_centers, 95)
    is_anomaly = dists_to_centers > threshold
    # Identifizieren der Indizes von Anomalien
    anomalous_indices = [i for i, flag in enumerate(is_anomaly) if flag]

    # Anomale Sequenzen ausgeben
    print("Indizes der Anomalien in den Testdaten:", anomalous_indices)
    num_anomalies = np.sum(is_anomaly)

    # Ergebnisse ausgeben
    print(f"Anzahl der Anomalien in den Testdaten: {num_anomalies}")

    # Plot für Trainingsdaten
    plt.figure(figsize=(10, 8))
    sns.scatterplot(x=latent_pca_train[:, 0], y=latent_pca_train[:, 1], hue=kmeans.labels_, palette="deep")
    plt.scatter(cluster_centers_pca[:, 0], cluster_centers_pca[:, 1], s=100, c='black', marker='*', label='Cluster Centers')
    plt.title("K-Means Clustering auf latenten Repräsentationen (Trainingsdaten)")
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.legend()
    wandb.log({"K-Means Clustering auf latenten Repräsentationen (Trainingsdaten)": wandb.Image(plt.gcf())})
    plt.show()

    # Plot für Testdaten
    plt.figure(figsize=(10, 8))
    sns.scatterplot(x=latent_pca_test[~is_anomaly, 0], y=latent_pca_test[~is_anomaly, 1], hue=test_cluster_labels[~is_anomaly], palette="deep", legend='full')
    sns.scatterplot(x=latent_pca_test[is_anomaly, 0], y=latent_pca_test[is_anomaly, 1], c='green', label='Anomalies', marker='X')
    plt.scatter(cluster_centers_pca[:, 0], cluster_centers_pca[:, 1], s=100, c='black', marker='*', label='Cluster Centers')
    plt.title("K-Means Clustering auf latenten Repräsentationen (Testdaten mit Anomalien)")
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.legend()
    wandb.log({"K-Means Clustering auf latenten Repräsentationen (Testdaten mit Anomalien)": wandb.Image(plt.gcf())})
    plt.show()



# Daten in Trainings-, Validierungs- und Testsets aufteilen
train_size = int(0.6 * len(tensor_sequences))
val_size = int(0.3 * len(tensor_sequences))
test_size = len(tensor_sequences) - train_size - val_size
train_set, val_set, test_set = random_split(tensor_sequences, [train_size, val_size, test_size])

train_loader = DataLoader(train_set, batch_size=64, shuffle=True,drop_last=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=True,drop_last=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=True, drop_last=True)

print("tensor_sequeces",len(tensor_sequences ))
print("train_size",train_size )
print("val_size", val_size)
print("test_size", test_size)


In [ ]:
# Hyperparameter
input_dim = 86  # Anzahl der Features
latent_dim = 46# Latente Dimension
learning_rate = 0.001
epochs = 15

#Trainingsumgebung und Modellinitialisierung
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
wandb.init(project='ae-model')
autoencoder = TransformerAutoencoder(input_dim, latent_dim).to(device)
wandb.watch(autoencoder)
optimizer = torch.optim.Adam(
    autoencoder.parameters(),
    lr=learning_rate

)
#Trainig des Modells
pretrain_autoencoder(autoencoder, train_loader, val_loader, optimizer, epochs)

#ONNX-Export
dummy_input = torch.randn(1, input_dim).to(device)  # Ersetzen Sie 'input_dim' durch die tatsächliche Eingabedimension
torch.onnx.export(autoencoder, dummy_input, "autoencoder.onnx")

# Hochladen des ONNX-Modells zu wandb
wandb.save("autoencoder.onnx")



In [ ]:
# Berechnen des durchschnittlichen Rekonstruktionsfehlers
train_error = calculate_average_reconstruction_error(autoencoder, train_loader, device='cuda' if torch.cuda.is_available() else 'cpu')
val_error = calculate_average_reconstruction_error(autoencoder, val_loader, device='cuda' if torch.cuda.is_available() else 'cpu')
test_error = calculate_average_reconstruction_error(autoencoder, test_loader, device='cuda' if torch.cuda.is_available() else 'cpu')
print(f"Durchschnittlicher Rekonstruktionsfehler - Training: {train_error}")
print(f"Durchschnittlicher Rekonstruktionsfehler - Validierung: {val_error}")
print(f"Durchschnittlicher Rekonstruktionsfehler - Testset: {test_error}")


In [ ]:
#DataLoader für Testdatensatz
anomaly_test_loader = DataLoader(test_set, batch_size=64, shuffle=False, drop_last=True)
#Anomalieerkennung mit Rekonstruktionsfehler
anomalies_identification_with_ReconstructionError(autoencoder, train_loader, anomaly_test_loader, device='cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#Anomalieerkennung mit K-means anhand latenten
anomalies_identification_with_Kmeans(train_loader, anomaly_test_loader, autoencoder)
wandb.finish()

In [ ]:
'''
 DEC Model
'''


class ClusterAssignmentLayer(nn.Module):
    def __init__(self, cluster_centers):
        super(ClusterAssignmentLayer, self).__init__()
        # Initialisierung der Clusterzentren als trainierbare Parameter
        self.cluster_centers = nn.Parameter(torch.Tensor(cluster_centers))

    def forward(self, x):
        # Berechnet die weiche Clusterzuweisung für die Eingabedaten x
        # Student's t-Verteilung wird zur Berechnung der Zuweisungswahrscheinlichkeiten verwendet
        q = 1.0 / (1.0 + (torch.sum((x.unsqueeze(1) - self.cluster_centers) ** 2, dim=2) / alpha))
        q = q ** ((alpha + 1.0) / 2.0)
        q = q / torch.sum(q, dim=1, keepdim=True)  # Normalisieren über alle Cluster für jede Probe
        return q


class DECModel(nn.Module):
    def __init__(self, autoencoder, cluster_centers):
        super(DECModel, self).__init__()
        # Einbettung des Autoencoders und der ClusterAssignmentLayer in das DEC-Modell
        self.autoencoder = autoencoder
        self.cluster_assignment = ClusterAssignmentLayer(cluster_centers)

    def forward(self, x):
        # Kodiert die Eingabe und erhält die latente Darstellung
        latent = self.autoencoder.encode(x)
         # Dekodiert die latente Darstellung zur Rekonstruktion der Eingabe
        reconstructed = self.autoencoder.decode(latent)
        # Reduziert die Dimensionalität
        if latent.dim() > 2:
            latent = latent.mean(dim=1)
        # Berechnet die weiche Clusterzuweisung für die latente Darstellung
        q = self.cluster_assignment(latent)
        return reconstructed, q


def target_distribution(q):
    # Berechnet die Zielfunktionsverteilung aus den weichen Clusterzuweisungen
    weight = q ** 2 / q.sum(0)
    # Normalisiert die Verteilung für jede Cluster-Komponente
    return (weight.t() / weight.sum(1)).t()

def kl_divergence_loss_function(q, p):
    # Berechnet die Kullback-Leibler-Divergenz zwischen den Verteilungen q und p
    kl_loss = F.kl_div(p.log(), q, reduction='batchmean')
    return kl_loss
# Verlustfunktion für die Rekonstruktion der Eingabedaten
def loss_function(recon_x, x, delta=1):
    recon_loss = F.smooth_l1_loss(recon_x, x, reduction='mean', beta=delta)
    return recon_loss
'''
# Clustering-Verlustfunktion
def clustering_loss_function(latent, cluster_centers):
    dists = torch.cdist(latent, cluster_centers)
    min_dists = torch.min(dists, dim=1).values
    return torch.mean(min_dists)

'''
# Haupttrainingsfunktion
def train_dec_model(dec_model, train_loader, val_loader, optimizer, epochs, alpha, gamma):
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        # Training
        dec_model.train()
        total_train_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            reconstructed, q = dec_model(data)

            # Berechnung des Rekonstruktionsverlusts
            recon_loss = loss_function(reconstructed, data)

            # Aktualisierung der Zielverteilung basierend auf den aktuellen weichen Zuweisungen q
            p = target_distribution(q).detach()
            # Berechnung des Clustering-Verlusts
            clustering_loss = kl_divergence_loss_function(q, p)

            # Gesamtverlust mit dem Balancierungsfaktor gamma
            loss = recon_loss + gamma * clustering_loss
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        train_losses.append(total_train_loss)

        # Validierung
        dec_model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for data in val_loader:
                data = data.to(device)
                reconstructed, q = dec_model(data)

                # Berechnung des Rekonstruktionsverlusts
                recon_loss = loss_function(reconstructed, data)

                # Keine Notwendigkeit, p während der Validierung zu aktualisieren, da wir nicht zurückpropagieren
                clustering_loss = kl_divergence_loss_function(q, target_distribution(q).detach())

                # Gesamtverlust für die Validierungsdaten
                loss = recon_loss + gamma * clustering_loss
                total_val_loss += loss.item()
        val_losses.append(total_val_loss)

        print(f"Epoch {epoch}, Train Loss: {total_train_loss}, Val Loss: {total_val_loss}")

    plot_train_val_loss(train_losses, val_losses)



def determine_threshold(distances, percentile=95):
  # Bestimmt den Schwellenwert für Anomalien basierend auf dem gegebenen Perzentil der Distanzen
    return np.percentile(distances, percentile)

def identify_anomalies(distances, threshold):
  # Identifiziert Anomalien, indem überprüft wird, ob Distanzen den Schwellenwert überschreiten
    return distances > threshold


def visualize_with_pca_and_anomalies(model, data_loader, cluster_centers, num_samples=1000, percentile=95):
    model.eval()  # Setzt das Modell in den Evaluierungsmodus
    latent_space = []  # Speichert latente Darstellungen
    cluster_labels = []  # Speichert Clusterzuweisungen
    distances_to_center = []  # Speichert Distanzen zu den Clusterzentren

    with torch.no_grad():  # Deaktiviert Gradientenberechnungen
        for data in data_loader:
            data = data.to(device)  # Verschiebt Daten auf das Gerät (z.B. GPU)
            latent = model.autoencoder.encode(data)  # Kodiert Daten in den latenten Raum

            # Reduziert die Dimensionalität, falls erforderlich
            if latent.dim() > 2:
                latent = latent.mean(dim=1)

            q = model.cluster_assignment(latent)  # Berechnet weiche Clusterzuweisungen
            cluster_label = torch.argmax(q, dim=1).cpu().numpy()  # Bestimmt den wahrscheinlichsten Cluster
            dists = torch.cdist(latent, cluster_centers).cpu().numpy()  # Berechnet Distanzen zu den Clusterzentren
            min_dists = np.min(dists, axis=1)  # Bestimmt minimale Distanz zu Clusterzentren

            # Sammelt Informationen für die Analyse
            latent_space.append(latent.cpu().numpy())
            cluster_labels.append(cluster_label)
            distances_to_center.append(min_dists)

            # Begrenzt die Anzahl der analysierten Samples
            if len(latent_space) >= num_samples:
                break

    # Konvertiert gesammelte Listen in Arrays und beschränkt sie auf die gewünschte Anzahl von Samples
    latent_space = np.concatenate(latent_space)[:num_samples]
    cluster_labels = np.concatenate(cluster_labels)[:num_samples]
    distances_to_center = np.concatenate(distances_to_center)[:num_samples]

    # Bestimmt den Schwellenwert für Anomalien und identifiziert Anomalien
    threshold = determine_threshold(distances_to_center, percentile=percentile)
    anomalies = identify_anomalies(distances_to_center, threshold)
    num_anomalies = np.sum(anomalies)
    print(f"Anzahl der Anomalien: {num_anomalies}")

    # Findet Indizes der Anomalien und druckt sie aus
    anomaly_indices = np.where(anomalies)[0]
    print("Indizes der Anomalien:", anomaly_indices)

    # Berechnet den Silhouetten-Score, falls mehr als ein Cluster vorhanden ist
    if len(np.unique(cluster_labels)) > 1:
        silhouette_avg = silhouette_score(latent_space, cluster_labels)
        print(f"Silhouetten-Score: {silhouette_avg}")
    else:
        print("Silhouetten-Score kann nicht berechnet werden, da nur ein Cluster vorhanden ist.")

    # Führt PCA durch, um die Daten auf zwei Dimensionen zu reduzieren
    all_points = np.vstack([latent_space, cluster_centers.cpu().numpy()])
    pca_results = PCA(n_components=2).fit_transform(all_points)

    # Visualisiert die Ergebnisse mit Matplotlib
    plt.figure(figsize=(10, 8))
    # Plottet die Datenpunkte, gruppiert nach Clustern
    for i in np.unique(cluster_labels):
        plt.scatter(pca_results[:-len(cluster_centers)][cluster_labels == i, 0], pca_results[:-len(cluster_centers)][cluster_labels == i, 1], alpha=0.5, label=f'Cluster {i}')
    # Plottet die Clusterzentren und Anomalien
    plt.scatter(pca_results[-len(cluster_centers):, 0], pca_results[-len(cluster_centers):, 1], c='black', marker='x', label='Clusterzentren', s=100)
    plt.scatter(pca_results[:-len(cluster_centers)][anomalies, 0], pca_results[:-len(cluster_centers)][anomalies, 1], c='red', alpha=0.5, label='Anomalien')
    # Fügt Titel, Beschriftungen und Legende hinzu
    plt.title('Clusterzuordnung und Anomalieerkennung im latenten Raum')
    plt.xlabel('PCA Feature 1')
    plt.ylabel('PCA Feature 2')
    plt.legend()
    # Loggt das Bild in wandb
    wandb.log({"Anomalien Identifikation": wandb.Image(plt.gcf())})
    plt.show()


# Daten in Trainings-, Validierungs- und Testsets aufteilen
train_size = int(0.6 * len(tensor_sequences))
val_size = int(0.3 * len(tensor_sequences))
test_size = len(tensor_sequences) - train_size - val_size
train_set, val_set, test_set = random_split(tensor_sequences, [train_size, val_size, test_size])

train_loader = DataLoader(train_set, batch_size=64, shuffle=True,drop_last=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=True,drop_last=True)
#test_loader = DataLoader(test_set, batch_size=64, shuffle=True, drop_last=True)


In [ ]:
# DEC-Modell mit initialisierten Clusterzentren
cluster_centers = torch.load('cluster_centers.pt').to(device)
alpha = 0.5# Freiheitsgrade der Student's t-Verteilung
gamma = 1
epochen=20

wandb.init(project='DEC-model')
dec_model = DECModel(autoencoder, cluster_centers).to(device)
wandb.watch(dec_model)
optimizer = torch.optim.Adam(
    dec_model.parameters(),
    lr=0.001
)


# Trainieren des DEC-Modells
train_dec_model(dec_model, train_loader, val_loader, optimizer, epochen, alpha=alpha,gamma=gamma)


#ONNX-Export
dummy_input = torch.randn(1, input_dim).to(device)  # Ersetzen Sie 'input_dim' durch die tatsächliche Eingabedimension
torch.onnx.export(dec_model, dummy_input, "DEC.onnx")

# Hochladen des ONNX-Modells zu wandb
wandb.save("DEC.onnx")


In [ ]:
# Erstellen eines DataLoader für den Testdatensatz zur Anomalieerkennung
#anomaly_test_loader = DataLoader(test_set, batch_size=64, shuffle=False, drop_last=True)
# Visualisierung der Anomalien mit PCA auf den Daten aus dem Testdatensatz
visualize_with_pca_and_anomalies(dec_model, anomaly_test_loader, cluster_centers, percentile=95)
wandb.finish()